In [2]:
import pandas as pd
df = pd.read_excel('/Users/souadmouajel/Desktop/Ironhack/lab-sessions/week-2/project-week2/GSAF5.xls')
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-06-17 00:00:00,2025.0,Unprovoked,USA,South Carolina,Hilton Head Island,Swimming,Not stated,F,12,...,Not stated,Kevin McMurray Trackingsharks.com:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-11 00:00:00,2025.0,Unprovoked,USA,Florida,Boca Grande,Snorkeling,Leah Lendel,F,9,...,Bull shark,Kevin McMurray Trackingsharks.com: James Kings...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-05-29 00:00:00,2025.0,Unprovoked,USA,North Carolina,Sunset Beach,Swimming,Sean Barton,M,26,...,Not stated,Kevin McMurray Trackingsharks.com: Clay Crewel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-05-26 00:00:00,2025.0,Unprovoked,Vanuatu,South Santo,Espiitu Santo Island,Swimming,Tumas,M,14,...,Not stated,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-05-15 00:00:00,2025.0,Unprovoked,Australia,South Australia,Port Noarlunga,Swimming,Richard Vinall,M,66,...,Not stated,Simon DeMarchi: Todd Smith: 9 News:ABC News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            7020 non-null   object 
 1   Year            7018 non-null   float64
 2   Type            7002 non-null   object 
 3   Country         6970 non-null   object 
 4   State           6535 non-null   object 
 5   Location        6454 non-null   object 
 6   Activity        6435 non-null   object 
 7   Name            6801 non-null   object 
 8   Sex             6441 non-null   object 
 9   Age             4026 non-null   object 
 10  Injury          6985 non-null   object 
 11  Fatal Y/N       6459 non-null   object 
 12  Time            3494 non-null   object 
 13  Species         3889 non-null   object 
 14  Source          7001 non-null   object 
 15  pdf             6799 non-null   object 
 16  href formula    6794 non-null   object 
 17  href            6796 non-null   o

In [6]:
# Challenges of Time feature: 
# 1 . alot of missing values
print(f"sum_of_missing_values: {df["Time"].isnull().sum()}")
print()
missing_percent = df["Time"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%")
print ()
print(f"Huge format variations: see the last 50 value counts") 
df["Time"].value_counts().tail(50)


sum_of_missing_values: 3526

Percentage of missing values: 50.23%

Huge format variations: see the last 50 value counts


Time
14h21                             1
15h53                             1
"Just before 11h00"               1
11h115                            1
12h39                             1
                                  1
N                                 1
Just before sundown               1
22h30                             1
11h30                             1
06h10                             1
Between 05h00 and 08h00           1
07h08                             1
17h00 or 17h40                    1
>08h00                            1
07h56                             1
15h56                             1
0830                              1
17h46                             1
21h50                             1
19h00, Dusk                       1
15h01                             1
1000                              1
10h44                             1
13h19                             1
Shortly before 12h00              1
17h34                             1
9h00                   

In [8]:
# Two steps clean and transform the Time featuers
# First, define a function to replace strange formates with valid ones using reg method (valid method can be strin like Late Afternoon, keep the valid ones 
# but null values have'nt replaced because they are a lot so we avoided the bias"
import pandas as pd
import re

def clean_time(value):
    if pd.isna(value):
        return None

    value = str(value).strip().lower()

    # Remove useless or unclear values
    if value in ["?", "am", "pm", "unknown", "not stated", "n/a", "na"]:
        return None

    # Clean formats like "after 1200hr", "11.30hr", "15.5", etc.
    match = re.search(r'(\d{1,2})[h:.]?(\d{2})?', value)

    if match:
        hour = match.group(1)
        minute = match.group(2) if match.group(2) else "00"
        return f"{hour.zfill(2)}:{minute.zfill(2)}"

    # Keep known phrases like "Morning", "Afternoon", etc.
    return value.title()
from datetime import datetime

# clean time results:
df['Cleaned_Time'] = df['Time'].apply(clean_time)
print(df['Cleaned_Time'].tail(60))

6960         Afternoon
6961         Afternoon
6962              None
6963              None
6964              None
6965              None
6966              None
6967              None
6968              None
6969              None
6970              None
6971              None
6972              None
6973              None
6974              None
6975              None
6976              None
6977    Late Afternoon
6978              None
6979              None
6980              None
6981              None
6982              None
6983         Afternoon
6984              None
6985              None
6986              None
6987              None
6988              None
6989              None
6990              None
6991              None
6992              None
6993              None
6994              None
6995              None
6996              None
6997              None
6998              None
6999              None
7000              None
7001              None
7002              None
7003       

In [10]:
# Second is to transform the inputs of so the datetime. 
# New function will be developed then it will applied on Time but the result will be saved on new feature called later Day_Part 
from datetime import datetime

def precise_time_to_day_part(value):
    if value is None:
        return None  # Keep missing as None

    # Known descriptive phrases to keep untouched
    descriptive_parts = [
        "Early Morning", "Morning", "Midday", "Early Afternoon",
        "Late Afternoon", "Afternoon", "Evening", "Dusk",
        "Night", "Late Night"
    ]
    
    if isinstance(value, str) and value.title() in descriptive_parts:
        return value.title()

    try:
        # Try parsing standard time like "14:30"
        time = datetime.strptime(value, "%H:%M").time()
        hour = time.hour
        minute = time.minute
        if 5 <= hour < 8:
            return "Early Morning"
        elif 8 <= hour < 12:
            return "Morning"
        elif hour == 12 and minute == 0:
            return "Midday"
        elif 12 <= hour < 15:
            return "Early Afternoon"
        elif 15 <= hour < 17:
            return "Late Afternoon"
        elif 17 <= hour < 19:
            return "Evening"
        elif 19 <= hour < 20:
            return "Dusk"
        elif 20 <= hour < 24:
            return "Night"
        else:  # 00:00 to before 5:00
            return "Late Night"
    except:
        return None  # Unrecognized values go to None
df['Cleaned_Time'] = df['Time'].apply(clean_time)
df['Day_Part'] = df['Cleaned_Time'].apply(precise_time_to_day_part)
print(df['Day_Part'].value_counts())

Day_Part
Morning            940
Early Afternoon    634
Late Afternoon     611
Evening            415
Afternoon          215
Early Morning      182
Midday             139
Night              125
Dusk                79
Late Night          38
Name: count, dtype: int64


In [12]:
# Challenges of Date feature: 
# 1 . There are no of missing values but we have to transform the values
print(f"sum_of_missing_values: {df["Date"].isnull().sum()}")
print()
missing_percent = df["Date"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%")
print ()
print(f"Huge format variations: see the last 50 value counts") 
df["Date"].value_counts().tail(50)

sum_of_missing_values: 0

Percentage of missing values: 0.00%

Huge format variations: see the last 50 value counts


Date
13-May-2000              1
Early Jun-2000           1
02-Jun-2000              1
10-Jun-2000              1
13-Jun-2000              1
19-Jun-2000              1
11-Sep-2000              1
15-Sep-2000              1
03-May-2001              1
03-Mar-2001              1
09-Jan-2001              1
21-Jan-2001              1
24-Jan-2001              1
Reported  24-Jan-2001    1
04-Feb-2001              1
11-Feb-2001              1
26-Feb-2001              1
Mar-2001                 1
08-Mar-2001              1
19-Sep-2000              1
09-Mar-2001              1
23-Mar-2001              1
02-Apr-2001              1
02-Ap-2001               1
05-Apr-2001              1
10-Apr-2001              1
28-Apr-2001              1
May-2001                 1
06-Jan-2001              1
24-Dec-2000              1
12-Dec-2000              1
11-Dec-2000              1
24-Sep-2000              1
25-Sep-2000              1
29-Sep-2000              1
02-Oct-2000              1
09-Oct-2000            

In [14]:
# In order to transform the Date column first we have to make sure "Date" column is in datetime format,
# So Text like "Unknown"; Dates missing month or day (e.g., "2023" or "March 2023"), Any other invalid format
df['Cleaned_Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Second we have to define the lables, we choose range date lables rather than season ones becuase the seasons can be different accoring to the country
def get_date_range_label(date):
    if pd.isna(date):
        return "No Date"
    
    day_of_year = date.timetuple().tm_yday  # Day number within the year

    # Margins based on the year of the date
    spring_start = datetime(date.year, 3, 21).timetuple().tm_yday
    summer_start = datetime(date.year, 6, 21).timetuple().tm_yday
    autumn_start = datetime(date.year, 9, 21).timetuple().tm_yday
    winter_start = datetime(date.year, 12, 21).timetuple().tm_yday

    if spring_start <= day_of_year < summer_start:
        return "21-03 to 21-06"
    elif summer_start <= day_of_year < autumn_start:
        return "21-06 to 21-09"
    elif autumn_start <= day_of_year < winter_start:
        return "21-09 to 21-12"
    else:
        return "21-12 to 21-03"

# Apply
df['Date_Range_Label'] = df['Cleaned_Date'].apply(get_date_range_label)

print(df[['Date', 'Cleaned_Date', 'Date_Range_Label']].head(5))
print ()
date_range_missing_values = df['Date_Range_Label'].value_counts().head(10)
print (f"Note: that we have now in Date_Range_Label some missing values {date_range_missing_values} called No Date")

                  Date Cleaned_Date Date_Range_Label
0  2025-06-17 00:00:00   2025-06-17   21-03 to 21-06
1  2025-06-11 00:00:00   2025-06-11   21-03 to 21-06
2  2025-05-29 00:00:00   2025-05-29   21-03 to 21-06
3  2025-05-26 00:00:00   2025-05-26   21-03 to 21-06
4  2025-05-15 00:00:00   2025-05-15   21-03 to 21-06

Note: that we have now in Date_Range_Label some missing values Date_Range_Label
21-06 to 21-09    1868
21-12 to 21-03    1652
21-03 to 21-06    1334
21-09 to 21-12    1290
No Date            876
Name: count, dtype: int64 called No Date


In [16]:
# Challenges of Injury feature: 
# 1 . There are few of missing values but variance is very high
# Challenges of Injury feature:
print(f"Sum of missing values: {df['Injury'].isnull().sum()}\n")

missing_percent = df["Injury"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%\n")

print("Huge format variations: see the last 50 value counts")
counts = df["Injury"].value_counts()
print(counts.tail(50))

print("\n")

# Filter where count is exactly 1
single_occurrences = counts[counts == 1]

# Show how many have count = 1
print(f"Number of unique injuries that appear only once: {len(single_occurrences)}")


Sum of missing values: 35

Percentage of missing values: 0.50%

Huge format variations: see the last 50 value counts
Injury
Left ankle & foot lacerated                                                                                                                  1
Left arm severed below shoulder                                                                                                              1
Laceration on right ankle & heel                                                                                                             1
Fatal, drowning or scavenging.  Two hours later his body, with bite marks,  washed ashore.                                                   1
Minor puncture wounds in hand                                                                                                                1
5 to 6 lacerations on right foot & ankle                                                                                                     1
Left thigh abraded

In [18]:
# In order to clean column Injury we went in two steps first:
# Purpose:
# Clean the "Injury" column text so that entries with the same injury but different cases, extra spaces, or special characters are standardized.
# for example: Lowercase: "Sprain" and "sprain" should be treated the same.
# Also Strip: Removes accidental spaces around text (" sprain ", "sprain ").
# Replace: Remove anything that is not a lowercase letter or space — this deletes numbers, punctuation, symbols, etc.
# For example, "Sprain#1" → "sprain"
# So we difned new column called Injury_clean using reex technic
df["Injury_clean"] = (
    df["Injury"]
    .str.lower()
    .str.strip()
    .str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-letter characters
)

# Quick grouping preview
print(df["Injury_clean"].value_counts())

Injury_clean
fatal                                                              882
foot bitten                                                        100
survived                                                            97
no injury                                                           94
leg bitten                                                          84
                                                                  ... 
punctures on left foot and foot                                      1
several  puncture wounds on lower right leg                          1
heel  foot bitten                                                    1
fatal lower thigh  knee severely lacerated                           1
fatal shark bit him in half carrying away the lower extremities      1
Name: count, Length: 4014, dtype: int64


In [20]:
# Second step we define new function called simplify_injury to recategorize all the the values in 7 categories,
# and generated new column called Injury grouped
def simplify_injury(text):
    if pd.isna(text):
        return "unknown"
    text = text.lower()
    if "fatal" in text:
        return "fatal"
    elif "foot" in text:
        return "foot injury"
    elif "leg" in text:
        return "leg injury"
    elif "hand" in text:
        return "hand injury"
    elif "no injury" in text:
        return "no injury"
    else:
        return "other"

df["Injury_grouped"] = df["Injury"].apply(simplify_injury)
print(df["Injury_grouped"].value_counts())

Injury_grouped
other          2481
fatal          1434
leg injury      902
foot injury     864
no injury       856
hand injury     448
unknown          35
Name: count, dtype: int64


In [22]:
print(f"Sum of missing values: {df['Age'].isnull().sum()}\n")

missing_percent = df["Age"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%\n")

print("Huge format variations: see the last 50 value counts")
counts = df["Age"].value_counts()
print(counts.tail(50))

print("\n")

Sum of missing values: 2994

Percentage of missing values: 42.65%

Huge format variations: see the last 50 value counts
Age
40?               1
45 and 15         1
12 or 13          1
18 or 20          1
86                1
!2                1
73                1
28 & 26           1
M                 1
74                1
!6                1
!!                1
4                 1
Elderly           1
28 & 22           1
18 months         1
22, 57, 31        1
20's              1
67                1
74                1
9 & 60            1
a minor           1
72                1
66                1
46 & 34           1
28, 23 & 30       1
Teens             1
77                1
87                1
16 to 18          1
32 & 30           1
69                1
60's              1
20?               1
 28               1
7      &    31    1
 30               1
60+               1
23 & 20           1
mid-30s           1
33 or 37          1
21 & ?            1
6½                1
30 or 36        

In [24]:
# clean the Age column by standarize the inputs to get integers:
import pandas as pd
import numpy as np

# Step 1: Define a function to clean each Age value and extract an integer or np.nan
def clean_age(age):
    if pd.isnull(age):
        return np.nan
    
    age_str = str(age).lower().strip()  # lowercase and strip whitespace
    
    # Handle empty strings
    if age_str == '' or age_str in ['unknown', 'n/a', 'na', '?']:
        return np.nan
    
    # Remove common unwanted characters
    for ch in ['?', '!', ',', '&', '-', '/', '\'']:
        age_str = age_str.replace(ch, ' ')
    
    # Handle special words
    if any(word in age_str for word in ['elderly', 'middle age', 'teen', 'minor', 'months']):
        return np.nan  # or you can assign special codes later
    
    # Extract all numbers in the string
    numbers = [int(s) for s in age_str.split() if s.isdigit()]
    
    if len(numbers) == 0:
        return np.nan
    
    # If multiple numbers, take average (or median)
    return int(round(sum(numbers) / len(numbers)))

# Apply the function to your Age column and create a new cleaned column
df['Age_clean'] = df['Age'].apply(clean_age)

print(df[['Age', 'Age_clean']].head(10))



  Age  Age_clean
0  12       12.0
1   9        9.0
2  26       26.0
3  14       14.0
4  66       66.0
5  26       26.0
6  45       45.0
7  20       20.0
8  30       30.0
9   ?        NaN


In [26]:
print(f"Sum of missing values: {df['Age_clean'].isnull().sum()}\n")

missing_percent = df["Age_clean"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%\n")

print("Huge format variations: see the last 50 value counts")
counts = df["Age_clean"].value_counts()
print(counts.tail(50))

print("\n")

Sum of missing values: 3082

Percentage of missing values: 43.90%

Huge format variations: see the last 50 value counts
Age_clean
9.0     46
41.0    45
39.0    44
45.0    42
50.0    42
52.0    40
46.0    36
8.0     36
49.0    35
48.0    33
47.0    33
44.0    32
55.0    29
51.0    28
7.0     27
57.0    26
60.0    24
58.0    23
54.0    21
6.0     17
59.0    17
61.0    17
53.0    17
56.0    16
62.0    13
69.0    11
68.0    11
63.0    10
64.0    10
70.0    10
65.0     9
66.0     8
5.0      7
3.0      5
73.0     5
77.0     4
71.0     4
75.0     4
74.0     3
67.0     3
4.0      2
1.0      2
78.0     2
82.0     1
72.0     1
2.0      1
86.0     1
84.0     1
87.0     1
81.0     1
Name: count, dtype: int64




In [28]:
# Transform the age to Age_group:
def assign_age_group(age):
    if pd.isnull(age):
        return 'Unknown'
    elif age < 13:
        return 'Child'
    elif 13 <= age < 20:
        return 'Teen'
    elif 20 <= age < 65:
        return 'Adult'
    else:
        return 'Senior'

df['Age_group'] = df['Age_clean'].apply(assign_age_group)

print(df[['Age_clean', 'Age_group']].value_counts())


Age_clean  Age_group
17.0       Teen         175
18.0       Teen         159
20.0       Adult        158
15.0       Teen         157
19.0       Teen         154
                       ... 
81.0       Senior         1
82.0       Senior         1
84.0       Senior         1
86.0       Senior         1
87.0       Senior         1
Name: count, Length: 82, dtype: int64


In [30]:
# HIPOLITO PART

#CLEAN THE FATAL COlUMN:


# Check the unique values in the column 'Fatal Y/N':
print(df['Fatal Y/N'].dropna().apply(lambda x: repr(x)).unique())


["'N'" "'Y'" "'F'" "'M'" "'n'" "'Nq'" "'UNKNOWN'" '2017' "'Y x 2'" "' N'"
 "'N '" "'y'"]


In [32]:
#Change to string and eliminate spaces
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()
#Check for valid values
valid_values = {'Y': 'Y', 'N': 'N'}
#Put all the good values the rest will be NaN
df['Fatal Y/N'] = df['Fatal Y/N'].map(valid_values)
#Result
print(df['Fatal Y/N'].unique())

['N' 'Y' nan]


In [34]:
# Count of values (Y, N)
print(df['Fatal Y/N'].value_counts(dropna=False))

Fatal Y/N
N      4897
Y      1480
NaN     643
Name: count, dtype: int64


In [36]:
# Sumayia Part
import numpy as np 

df['Location'] = df['Location'].replace({
    'Panama Bay 8ºN, 79ºW': 'Panama Bay'
}) 
print(df['Location'])

0                      Hilton Head Island  
1                               Boca Grande
2                              Sunset Beach
3                      Espiitu Santo Island
4                            Port Noarlunga
                       ...                 
7015                            Roebuck Bay
7016                                    NaN
7017                         Ocracoke Inlet
7018                             Panama Bay
7019    Below the English fort, Trincomalee
Name: Location, Length: 7020, dtype: object


In [38]:
def clean_column_location(df,column='Location'):
    df[column] = df[column].str.strip().str.upper()
    df[column] = df[column].replace(df['Location']).unique
    return df

df = clean_column_location(df,column='Location')
print(sorted(df['Location'].dropna().unique()))

[<bound method Series.unique of 0                        HILTON HEAD ISLAND
1                               BOCA GRANDE
2                              SUNSET BEACH
3                      ESPIITU SANTO ISLAND
4                            PORT NOARLUNGA
                       ...                 
7015                            ROEBUCK BAY
7016                                    NaN
7017                         OCRACOKE INLET
7018                             PANAMA BAY
7019    BELOW THE ENGLISH FORT, TRINCOMALEE
Name: Location, Length: 7020, dtype: object>]


In [39]:
df.State.unique()


array(['South Carolina', 'Florida', 'North Carolina', 'South Santo',
       'South Australia', 'Montego Bay', 'Hadera', 'Quntana Roo',
       'Kulhudhuffushi City', 'WA', 'NSW', 'Victoria', 'Batangas Resort',
       'Western Australia', 'Queensland', nan,
       'Near five Cays settlement West Caicos', 'Inhambane Province',
       'Hawaii', 'Grande Terre', '?', 'North of Marsa Alam',
       'Phang Nga Province', 'Bay of Waitangi', 'Kaafu', 'Maui', 'Tela',
       'Timor Leste', 'Trujillo Colon', 'Southern Morocco',
       'Gulf of Honduras', 'Texas', 'Floria', 'California',
       'Maahvah Laamu Atoll', 'Galveston', 'Providenciales Island',
       'Gambier Islands', 'Graet Courland Bay',
       'Clarence Town Long Isand', 'Nunu Atoll', 'Maharashtra',
       'New  South Wales', 'Paradise Island', 'Eastern Cape Province',
       'Sonora', 'New South Wales', 'South Island', 'Guerrero',
       'New Providence   Isoad', 'Jalisco', 'Grand  Bahama Island',
       'Westerm Australia', 'North Is

In [42]:
df['State'].fillna(method='ffill').tail()


/var/folders/tn/xjfvdsqn1_d879f1bppyx1xh0000gn/T/ipykernel_1151/3441788617.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['State'].fillna(method='ffill').tail()


7015    Western Australia
7016    Western Australia
7017       North Carolina
7018       North Carolina
7019     Eastern Province
Name: State, dtype: object

In [44]:
import numpy as np



State = {
    'Floria': 'Florida',
    'South Carolina ': 'South Carolina',
    'North Carolina ': 'North Carolina',
    'New  South Wales': 'New South Wales',
    'New South ales': 'New South Wales',
    'New South Wales ': 'New South Wales',
    'Baja ': 'Baja California',
    'Westerm Australia': 'Western Australia',
    'Maahvah Laamu Atoll': 'Laamu Atoll',
    'Grand  Bahama Island': 'Grand Bahama Island',
    'Isla De San Andres': 'San Andrés Island',
    'Lucayan Lucayan Archipelago': 'Lucayan Archipelago',
    'New Providence   Isoad': 'New Providence Island',
    'Hurghada, Red Sea Governorate': 'Red Sea Governorate',
    'KwaZulu-Natal between Port Edward and Port St Johns': 'KwaZulu-Natal',
    'Western  Australia': 'Western Australia',
    'Western Cape Province': 'Western Cape',
    'Noirth Carolina': 'North Carolina',
    'Guerro': 'Guerrero',
    'Guerrrero': 'Guerrero',
    'Namonuito Atoll': 'Micronesia',
    'Grand Baie': 'Mauritius',
    'Guantanamo Province': 'Guantánamo Province',
    'Unknown, treated at Wick, SCOTLAND': 'Unknown',
    'Bahamas': 'The Bahamas',
    'BAHAMAS': 'The Bahamas',
    'Exumas': 'Exuma Islands',
    'Grand Bahama Island': 'Grand Bahama Island',
    ' Grand Bahama Island': 'Grand Bahama Island',
    'South Santo': 'Espírito Santo',
    'Montego Bay': 'Jamaica',
    'Grande Terre': 'New Caledonia',
    '?': 'Unknown',
    'nan': 'Unknown',
    'Lucayan Lucayan Archipelago': 'Lucayan Archipelago',
    'South Province': 'Unknown',
    'KNZ': 'KwaZulu-Natal', 
    'New South ales': 'New South Wales',
    'Noirth Carolina': 'North Carolina',
    'KZN':'KwaZulu-Natal', 
    '40 miles off Grand Bahama Island': 'Unknown', 
    '740 miles SE of Tarawa Atoll':'Unknown',
    '300 miles from Antigua': 'Unknown',
    '800 miles from land': 'Unknown',
    '600 nm west of the Canary Islands': 'Unknown', 
    'KwaZulu-Natal between Port Edward and Port St Johns': 'Unknown',
    '12 miles off the north coast': 'Unknown',
    'New Territories': 'Unknown',
    'On the Kowloon penisula, south of Sai Kung': 'Unknown',
    'Between DR and Puerto Rico': 'Unknown', 
    'Between Honiara & Isabel Island': 'Unknown',
    "Ha'api": 'Unknown', 
    'South China Sea 200 miles from Hong Kong': 'Unknown',
    '200 nm southeast of Manila': 'Unknown', 
    "250 miles southwest of O'ahu, Hawaii": 'Unknown',
    'Near Bougainville (North Solomons)': 'Unknown', 
    'Off the Coromandel Peninsula, North Island': 'Unknown',
    '10ºS, 142ºE': 'Unknown', 
    '165  miles from Bermuda': 'Unknown',
    '25 km off the coast of Iran & 483km from mouth of Persian Gulf': 'Unknown',
    '19S, 178?E': 'Unknown', 
    '9.35N 79.35W': 'Unknown', 
    'Enroute from Suez to Aden (Yemen)': 'Unknown',
    '180 miles southeast of Okinawa': 'Unknown', 
    'In the English Channel ': 'Unknown',
    'Unknown, treated at Wick, SCOTLAND': 'Unknown',
    '33N, 68W': 'Unknown', 
    'Madang (WO)': 'Unknown', 
    'Between Timor & Darwin, Australia': 'Unknown',
    '400 miles southeast of Sri Lanka': 'Unknown',
    'In the Gulf Stream ': 'Unknown',
    'Between England & South Africa': 'Unknown', 
    'Mindanao': 'Unknown',
    'Between Hawaii & Wake Island': 'Unknown', 
    '1,000 miles east of Hawaii': 'Unknown', 
    'Central Province': 'Unknown',
    '1000 miles west of Hawaii': 'Unknown', 
    '18S / 50E': 'Unknown',
    '330 to 350 miles east of Wake Island': 'Unknown',
    'Between Kwajalein Atoll & Johnston Island': 'Unknown', 
    'In transit between Tinian and Leyte': 'Unknown',
    '300 miles east of Luzon': 'Unknown',
    'Bernardino Strait near Gulf of Leyte': 'Unknown',
    'Off Samar Island in the Gulf of Leyte': 'Unknown',
    'Lake Nicaragua (fresh water)': 'Unknown', 
    'Near the Fiji Islands': 'Unknown', 
    '40 miles south of Naples ': 'Unknown',
    'Northwest of Papua New Guinea': 'Unknown', 
    'Between Hawaii and U.S.A.': 'Unknown',
    'Off South American coast': 'Unknown',
    '04.05N-13.23W': 'Unknown', 
    '300 miles east of St. Thomas (Virgin Islands)': 'Unknown',
    'West of Ceylon (Sri  Lanka)': 'Unknown', 
    'Off Libya': 'Unknown',  
    'North of Pernambuco, Brazil': 'Unknown', 
    'In Convoy OB 274': 'Unknown', 
    '2 to 3 miles off Taboguilla Island, Pacific Ocean': 'Unknown', 
    '150 miles offshore': 'Unknown', 
    '60 miles north of San Domingo in the West Indies': 'Unknown', 
    '30 nm from Singapore': 'Unknown',
    'Somewhere between Philadelphia and Hiogo, Japan': 'Unknown', 
    'Between Hastings & Fairlight, Sussex': 'Unknown', 
    'Off the coast of South America': 'Unknown', 
    '22ºN, 88ºE': 'Unknown',
    '300 miles east of Mauritius': 'Unknown',
    'Between Australia & USA': 'Unknown', 
    "35º39 : 165º8'": 'Unknown', 
    'Between New Ireland & New Britain': 'Unknown'
}  


df['state_clean'] = df['State'].str.strip()

# Apply mapping
df['state_standardized'] = df['state_clean'].replace('State')

# Optional: Replace all nulls or ambiguous with 'Unknown'
df['state_standardized'] = df['state_standardized'].fillna('Unknown')

# Check unique cleaned states
cleaned_unique_state = df['state_standardized'].unique()

/var/folders/tn/xjfvdsqn1_d879f1bppyx1xh0000gn/T/ipykernel_1151/4131875704.py:120: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['state_standardized'] = df['state_clean'].replace('State')


In [46]:
def clean_column_state(df, column='State'):
    df[column] = df[column].str.strip().str.upper()
    df[column] = df[column].replace('state')
    return df

df = clean_column_state(df,column='State')
print(sorted(df['State'].dropna().unique()))

['"HEAD OF THE GULF"', '(SOUTHWESTERN PACIFIC)', '04.05N-13.23W', '1,000 MILES EAST OF HAWAII', '1000 MILES WEST OF HAWAII', '10ºS, 142ºE', '12 MILES OFF THE NORTH COAST', '150 MILES OFFSHORE', '165  MILES FROM BERMUDA', '180 MILES SOUTHEAST OF OKINAWA', '18S / 50E', '19S, 178?E', '2 TO 3 MILES OFF TABOGUILLA ISLAND, PACIFIC OCEAN', '200 NM SOUTHEAST OF MANILA', '22ºN, 88ºE', '25 KM OFF THE COAST OF IRAN & 483KM FROM MOUTH OF PERSIAN GULF', "250 MILES SOUTHWEST OF O'AHU, HAWAII", '30 NM FROM SINGAPORE', '300 MILES EAST OF LUZON', '300 MILES EAST OF MAURITIUS', '300 MILES EAST OF ST. THOMAS (VIRGIN ISLANDS)', '300 MILES FROM ANTIGUA', '330 TO 350 MILES EAST OF WAKE ISLAND', '33N, 68W', "35º39 : 165º8'", '40 MILES OFF GRAND BAHAMA ISLAND', '40 MILES SOUTH OF NAPLES', '400 MILES SOUTHEAST OF SRI LANKA', '5AINT-DENIS', '60 MILES NORTH OF SAN DOMINGO IN THE WEST INDIES', '600 NM WEST OF THE CANARY ISLANDS', '740 MILES SE OF TARAWA ATOLL', '800 MILES FROM LAND', '9.35N 79.35W', '?', 'ABACO I

/var/folders/tn/xjfvdsqn1_d879f1bppyx1xh0000gn/T/ipykernel_1151/1181233811.py:3: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df[column] = df[column].replace('state')


In [48]:
# Top 10 State with more shark attack
top_10_state = df['State'].value_counts().head(10)
print(top_10_state)

State
FLORIDA                  1188
NEW SOUTH WALES           523
QUEENSLAND                353
HAWAII                    344
CALIFORNIA                324
WESTERN AUSTRALIA         237
KWAZULU-NATAL             218
WESTERN CAPE PROVINCE     197
SOUTH CAROLINA            175
EASTERN CAPE PROVINCE     168
Name: count, dtype: int64


In [50]:
# Check the unique values in the column 'Country'
print(df['Country'].unique())

['USA' 'Vanuatu' 'Australia' 'Jamaica' 'Israel' 'Mexico' 'Maldives'
 'Philippines' 'Bahamas' 'Turks and Caicos' 'Mozambique' 'New Caledonia'
 'Egypt' 'Thailand' 'New Zealand' 'Hawaii' 'Honduras' 'Indonesia'
 'Morocco' 'Belize' 'Maldive Islands' 'French Polynesia' 'Tobago'
 'AUSTRALIA' 'INDIA' 'TRINIDAD' 'BAHAMAS' 'SOUTH AFRICA' 'MEXICO'
 'NEW ZEALAND' 'EGYPT' 'BELIZE' 'PHILIPPINES' 'Coral Sea' 'SPAIN'
 'PORTUGAL' 'SAMOA' 'COLOMBIA' 'ECUADOR' 'FRENCH POLYNESIA'
 'NEW CALEDONIA' 'TURKS and CaICOS' 'CUBA' 'BRAZIL' 'SEYCHELLES'
 'ARGENTINA' 'FIJI' 'MeXICO' 'South Africa' 'ENGLAND' 'JAPAN' 'INDONESIA'
 'JAMAICA' 'MALDIVES' 'THAILAND' 'COLUMBIA' 'COSTA RICA'
 'British Overseas Territory' 'CANADA' 'JORDAN' 'ST KITTS / NEVIS'
 'ST MARTIN' 'PAPUA NEW GUINEA' 'REUNION ISLAND' 'ISRAEL' 'CHINA'
 'IRELAND' 'ITALY' 'MALAYSIA' 'LIBYA' nan 'MAURITIUS' 'SOLOMON ISLANDS'
 'ST HELENA, British overseas territory' 'COMOROS' 'REUNION'
 'UNITED KINGDOM' 'UNITED ARAB EMIRATES' 'CAPE VERDE' 'Fiji'
 'DOMINICAN 

In [52]:
#All in mayus and eliminate spaces
df['Country'] = df['Country'].str.strip().str.upper()
print(df['Country'].unique())

['USA' 'VANUATU' 'AUSTRALIA' 'JAMAICA' 'ISRAEL' 'MEXICO' 'MALDIVES'
 'PHILIPPINES' 'BAHAMAS' 'TURKS AND CAICOS' 'MOZAMBIQUE' 'NEW CALEDONIA'
 'EGYPT' 'THAILAND' 'NEW ZEALAND' 'HAWAII' 'HONDURAS' 'INDONESIA'
 'MOROCCO' 'BELIZE' 'MALDIVE ISLANDS' 'FRENCH POLYNESIA' 'TOBAGO' 'INDIA'
 'TRINIDAD' 'SOUTH AFRICA' 'CORAL SEA' 'SPAIN' 'PORTUGAL' 'SAMOA'
 'COLOMBIA' 'ECUADOR' 'CUBA' 'BRAZIL' 'SEYCHELLES' 'ARGENTINA' 'FIJI'
 'ENGLAND' 'JAPAN' 'COLUMBIA' 'COSTA RICA' 'BRITISH OVERSEAS TERRITORY'
 'CANADA' 'JORDAN' 'ST KITTS / NEVIS' 'ST MARTIN' 'PAPUA NEW GUINEA'
 'REUNION ISLAND' 'CHINA' 'IRELAND' 'ITALY' 'MALAYSIA' 'LIBYA' nan
 'MAURITIUS' 'SOLOMON ISLANDS' 'ST HELENA, BRITISH OVERSEAS TERRITORY'
 'COMOROS' 'REUNION' 'UNITED KINGDOM' 'UNITED ARAB EMIRATES' 'CAPE VERDE'
 'DOMINICAN REPUBLIC' 'CAYMAN ISLANDS' 'ARUBA' 'PUERTO RICO'
 'ATLANTIC OCEAN' 'GREECE' 'ST. MARTIN' 'FRANCE' 'TRINIDAD & TOBAGO'
 'KIRIBATI' 'DIEGO GARCIA' 'TAIWAN' 'PALESTINIAN TERRITORIES' 'GUAM'
 'NIGERIA' 'TONGA' 'SCOTLAND' '

In [54]:
# CLEANING DATA OF COUNTRIES

import numpy as np

country_corrections = {
    # Correccions Ortografics
    'COLUMBIA': 'COLOMBIA',
    'TRINIDAD & TOBAGO': 'TRINIDAD AND TOBAGO',
    'MALDIVE ISLANDS': 'MALDIVES',
    'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES',
    'ST. MARTIN': 'ST MARTIN',
    'ST. MAARTIN': 'ST MARTIN',
    'TRINIDAD': 'TRINIDAD AND TOBAGO',

    # Agrupations
    'ENGLAND': 'UK',
    'SCOTLAND': 'UK',
    'UNITED KINGDOM': 'UK',
    'BRITISH ISLES': 'UK',
    'BRITISH WEST INDIES': 'UK',
    'BRITISH VIRGIN ISLANDS': 'UK',

    # Ocean y region not usefull
    'PACIFIC OCEAN': 'OTHER',
    'ATLANTIC OCEAN': 'OTHER',
    'INDIAN OCEAN': 'OTHER',
    'SOUTH PACIFIC OCEAN': 'OTHER',
    'CARIBBEAN SEA': 'OTHER',
    'OCEAN': 'OTHER',
    'GULF OF ADEN': 'OTHER',
    'MID-PACIFC OCEAN': 'OTHER',
    'NORTH ATLANTIC OCEAN': 'OTHER',
    'RED SEA': 'OTHER',
    'RED SEA / INDIAN OCEAN': 'OTHER',
    'NORTH PACIFIC OCEAN': 'OTHER',
    'CENTRAL PACIFIC': 'OTHER',

    # Some other mistakes → agrupar
    'DIEGO GARCIA': 'OTHER',
    'JOHNSTON ISLAND': 'OTHER',
    'ADMIRALTY ISLANDS': 'OTHER',
    'MID ATLANTIC OCEAN': 'OTHER',
    'UNKNOWN': 'OTHER',
    'AFRICA': 'OTHER',
    'ASIA?': 'OTHER',
    'SUDAN?': 'SUDAN',
}

In [56]:
# Import the country correction on or columns

def clean_column_country(df, column='Country'):
    # All mayus
    df[column] = df[column].str.strip().str.upper()
    #use the country_corrections to filter the column
    df[column] = df[column].replace(country_corrections)
    return df

df = clean_column_country(df, column='Country')

print(sorted(df['Country'].dropna().unique()))

['ALGERIA', 'AMERICAN SAMOA', 'ANDAMAN / NICOBAR ISLANDAS', 'ANDAMAN ISLANDS', 'ANGOLA', 'ANTIGUA', 'ARGENTINA', 'ARUBA', 'AUSTRALIA', 'AZORES', 'BAHAMAS', 'BAHREIN', 'BANGLADESH', 'BARBADOS', 'BAY OF BENGAL', 'BELIZE', 'BERMUDA', 'BETWEEN PORTUGAL & INDIA', 'BRAZIL', 'BRITISH NEW GUINEA', 'BRITISH OVERSEAS TERRITORY', 'BURMA', 'CANADA', 'CAPE VERDE', 'CAYMAN ISLANDS', 'CEYLON', 'CEYLON (SRI LANKA)', 'CHILE', 'CHINA', 'COAST OF AFRICA', 'COLOMBIA', 'COMOROS', 'COOK ISLANDS', 'CORAL SEA', 'COSTA RICA', 'CRETE', 'CROATIA', 'CUBA', 'CURACAO', 'CYPRUS', 'DJIBOUTI', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EGYPT', 'EGYPT / ISRAEL', 'EL SALVADOR', 'EQUATORIAL GUINEA / CAMEROON', 'FALKLAND ISLANDS', 'FEDERATED STATES OF MICRONESIA', 'FIJI', 'FRANCE', 'FRENCH POLYNESIA', 'GABON', 'GEORGIA', 'GHANA', 'GRAND CAYMAN', 'GREECE', 'GREENLAND', 'GRENADA', 'GUAM', 'GUATEMALA', 'GUINEA', 'GUYANA', 'HAITI', 'HAWAII', 'HONDURAS', 'HONG KONG', 'ICELAND', 'INDIA', 'INDIAN OCEAN?', 'INDONESIA', 'IRAN', 'IRAN / IRA

In [58]:
# Top 10 countries with more sharks attacts:
top_10_paises = df['Country'].value_counts().head(10)
print(top_10_paises)

Country
USA                 2561
AUSTRALIA           1502
SOUTH AFRICA         598
NEW ZEALAND          146
BAHAMAS              139
PAPUA NEW GUINEA     136
BRAZIL               122
MEXICO               107
OTHER                 83
ITALY                 72
Name: count, dtype: int64


In [61]:
# Damian Part
activity_counts = df['Activity'].value_counts()

# Get top 10
top_10 = activity_counts.head(10)

# Calculate sum of all other activities
other_count = activity_counts[10:].sum()

# Create a new series with "Other activities" included
top_10_with_other = pd.concat([
    top_10, 
    pd.Series({'Other activities': other_count})
])

print("Top 10 activities plus others:")
print(top_10_with_other)

print("\nFormatted table:")
print(top_10_with_other.to_markdown())

Top 10 activities plus others:
Surfing             1133
Swimming             997
Fishing              490
Spearfishing         388
Wading               177
Bathing              164
Diving               147
Snorkeling           132
Standing             113
Scuba diving          84
Other activities    2610
dtype: int64

Formatted table:
|                  |    0 |
|:-----------------|-----:|
| Surfing          | 1133 |
| Swimming         |  997 |
| Fishing          |  490 |
| Spearfishing     |  388 |
| Wading           |  177 |
| Bathing          |  164 |
| Diving           |  147 |
| Snorkeling       |  132 |
| Standing         |  113 |
| Scuba diving     |   84 |
| Other activities | 2610 |


In [63]:
import pandas as pd

def get_activity_stats(df, activity_column='Activity', top_n=10):

    # Make copy to avoid modifying original DataFrame
    df = df.copy()
    
    # Standard cleaning
    df[activity_column] = (
        df[activity_column]
        .str.strip()  # Remove whitespace
        .str.lower()  # Convert to lowercase
    )
    
    # Common activity replacements (customize as needed)
    activity_replacements = {
        'swim': 'swimming',
        'bike': 'cycling',
        'bicycle': 'cycling',
        'football': 'soccer',
        'bball': 'basketball',
        'hoops': 'basketball',
        # Add more as needed for your dataset
    }
    
    df[activity_column] = df[activity_column].replace(activity_replacements)
    
    # Get activity counts
    activity_counts = df[activity_column].value_counts().reset_index()
    activity_counts.columns = ['Activity', 'Count']
    total = activity_counts['Count'].sum()
    # Separate top N and others
    top_activities = activity_counts.head(top_n)
    other_count = activity_counts['Count'][top_n:].sum()
    
    # Create "Other" row
    other_row = pd.DataFrame({
        'Activity': ['Other activities'],
        'Count': [other_count],
        'Percentage': [(other_count / total * 100).round(1)]
    })
    
    # Calculate percentages for top activities
    top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)
    
    # Combine results
    result = pd.concat([top_activities, other_row], ignore_index=True)
    
    return result

# Usage example:
activity_stats = get_activity_stats(df, top_n=10)

print("Activity Statistics (Top 10 + Others):")
print(activity_stats.to_markdown(index=False, floatfmt=".1f"))

Activity Statistics (Top 10 + Others):
| Activity         |   Count |   Percentage |
|:-----------------|--------:|-------------:|
| surfing          |    1138 |         17.7 |
| swimming         |    1044 |         16.2 |
| fishing          |     506 |          7.9 |
| spearfishing     |     396 |          6.2 |
| wading           |     177 |          2.8 |
| bathing          |     167 |          2.6 |
| diving           |     150 |          2.3 |
| snorkeling       |     133 |          2.1 |
| standing         |     115 |          1.8 |
| scuba diving     |     104 |          1.6 |
| Other activities |    2505 |         38.9 |


/var/folders/tn/xjfvdsqn1_d879f1bppyx1xh0000gn/T/ipykernel_1151/2166415423.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)


In [65]:
def clean_sex_column(df, column_name='Sex'):
    """
    Cleans a sex column by:
    1. Converting all values to strings
    2. Stripping whitespace
    3. Converting to uppercase
    4. Keeping only 'M' or 'F'
    5. Setting all others to NaN
    """
    
    # Convert to string, strip whitespace, uppercase
    cleaned = df[column_name].astype(str).str.strip().str.upper()
    
    # Keep only M or F, others become NaN
    df[column_name] = cleaned.where(cleaned.isin(['M', 'F']))
    
    # Count cleaned values
    print("\nCleaning report:")
    print(f"Original value counts:\n{df[column_name].astype(str).str.strip().str.upper().value_counts()}")
    print(f"\nValid 'M' count: {(cleaned == 'M').sum()}")
    print(f"Valid 'F' count: {(cleaned == 'F').sum()}")
    print(f"Invalid values set to NaN: {len(df) - ((cleaned == 'M').sum() + (cleaned == 'F').sum())}")
    
    return df

df_cleaned = clean_sex_column(df, 'Sex')

print("\nFirst 10 rows after cleaning:")
print(df_cleaned[['Sex']].head(10))


Cleaning report:
Original value counts:
Sex
M      5634
F       802
NAN     584
Name: count, dtype: int64

Valid 'M' count: 5634
Valid 'F' count: 802
Invalid values set to NaN: 584

First 10 rows after cleaning:
  Sex
0   F
1   F
2   M
3   M
4   M
5   F
6   M
7   M
8   M
9   M


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                7020 non-null   object        
 1   Year                7018 non-null   float64       
 2   Type                7002 non-null   object        
 3   Country             6970 non-null   object        
 4   State               6535 non-null   object        
 5   Location            7020 non-null   object        
 6   Activity            6435 non-null   object        
 7   Name                6801 non-null   object        
 8   Sex                 6436 non-null   object        
 9   Age                 4026 non-null   object        
 10  Injury              6985 non-null   object        
 11  Fatal Y/N           6377 non-null   object        
 12  Time                3494 non-null   object        
 13  Species             3889 non-null   object      

In [69]:
# Check for Duplicated value but we found nothing
duplicates = df[df.duplicated()]

print(duplicates)

Empty DataFrame
Columns: [Date, Year, Type, Country, State, Location, Activity, Name, Sex, Age, Injury, Fatal Y/N, Time, Species , Source, pdf, href formula, href, Case Number, Case Number.1, original order, Unnamed: 21, Unnamed: 22, Cleaned_Time, Day_Part, Cleaned_Date, Date_Range_Label, Injury_clean, Injury_grouped, Age_clean, Age_group, state_clean, state_standardized]
Index: []

[0 rows x 33 columns]


In [71]:
# change the type of columns to store the data efficiantly
# for our analysis we will use the following columns:
# Fatal Y/N, state_standardized (transoformed from State), Injury_grouped (transoformed from Injury)... 
# ...Date_Range_Label (transoformed from Date), Day_Part (transoformed from Time), Age_group (Tranformed from Age), Sex, Activity, Location, Country
# Lets change some of the datatype of the columns to insure better storage efficiancy

In [73]:
df['state_standardized'] = df['state_standardized'].astype('string')
print(df['state_standardized'].dtype)

string


In [75]:
df['Injury_grouped'] = df['Injury_grouped'].astype('string')
print(df['Injury_grouped'].dtype)

string


In [77]:
df['Date_Range_Label'] = df['Date_Range_Label'].astype('string')
print(df['Date_Range_Label'].dtype)

string


In [79]:
df['Day_Part'] = df['Day_Part'].astype('string')
print(df['Day_Part'].dtype)

string


In [81]:
df['Age_group'] = df['Age_group'].astype('string') 
print(df['Age_group'].dtype)

string


In [83]:
df['Sex'] = df['Sex'].astype("string")
print(df['Sex'].dtype)

string


In [85]:
df['Activity'] = df['Activity'].astype('string')
print(df['Activity'].dtype)

string


In [378]:
# Don't run this
#df['Location'] = df['Location'].astype('string')
#print(df['Location'].dtype)

string


In [380]:
# Don't run this
#df['Country'] = df['Country'].astype('string')
#print(df['Country'].dtype)

string


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                7020 non-null   object        
 1   Year                7018 non-null   float64       
 2   Type                7002 non-null   object        
 3   Country             6970 non-null   object        
 4   State               6535 non-null   object        
 5   Location            7020 non-null   object        
 6   Activity            6435 non-null   string        
 7   Name                6801 non-null   object        
 8   Sex                 6436 non-null   string        
 9   Age                 4026 non-null   object        
 10  Injury              6985 non-null   object        
 11  Fatal Y/N           6377 non-null   object        
 12  Time                3494 non-null   object        
 13  Species             3889 non-null   object      

In [101]:
# Group by state and fatal status, state_standardized
grouped_state_fatal = df.groupby(['Fatal Y/N', 'state_standardized']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# First result of the analysis: clearly our tourist should consider that beaches of the following sate are highly risky: 
# New South Wales, Queensland, Hawaii, Florida, KwaZulu-Natal

    Fatal Y/N     state_standardized  Count
976         Y                Unknown    195
825         Y        New South Wales    104
883         Y             Queensland     82
717         Y                 Hawaii     50
686         Y                Florida     48
745         Y          KwaZulu-Natal     48
996         Y      Western Australia     36
998         Y  Western Cape Province     35
931         Y        South Australia     30
867         Y             Pernambuco     30
677         Y  Eastern Cape Province     24
969         Y          Torres Strait     23
637         Y             California     22
832         Y           North Island     14
983         Y               Victoria     14
830         Y         North Carolina     12
648         Y       Central Province     12
982         Y               Veracruz     12
716         Y        Havana Province     11
932         Y         South Carolina     11


In [107]:
# Group by state and fatal status, Date_Range_Label
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Date_Range_Label']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# Second: the Winter time of the north of earth is also very risky, but to know better we have to add the country factor that we will see later

  Fatal Y/N Date_Range_Label  Count
8         Y   21-12 to 21-03    380
9         Y          No Date    305
6         Y   21-06 to 21-09    300
7         Y   21-09 to 21-12    252
5         Y   21-03 to 21-06    243


In [109]:
# Group by state and fatal status, Day_Part
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Day_Part']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# between the morning and the end of the afternoon is the riskiest time

   Fatal Y/N         Day_Part  Count
18         Y          Morning    146
15         Y   Late Afternoon    107
12         Y  Early Afternoon    101
14         Y          Evening     64
19         Y            Night     30
10         Y        Afternoon     28
13         Y    Early Morning     24
17         Y           Midday     24
16         Y       Late Night     11
11         Y             Dusk      6


In [113]:
# Group by state and fatal status, Age_group
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Age_group']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# Adults must be more wise since they are more likely to be attacked by sharks

  Fatal Y/N Age_group  Count
9         Y   Unknown    794
5         Y     Adult    437
8         Y      Teen    191
6         Y     Child     37
7         Y    Senior     21


In [115]:
# Group by state and fatal status, Sex
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Sex']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# Males are more likely to be attacked

  Fatal Y/N Sex  Count
3         Y   M   1249
2         Y   F    122


In [127]:
# Group by state and fatal status, Activity
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Activity']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# Swimming, Bathing, Surfing ate top three must Dangerous activities

     Fatal Y/N             Activity  Count
1401         Y             Swimming    324
1046         Y              Bathing     71
1395         Y              Surfing     64
1367         Y         Spearfishing     53
1177         Y              Fishing     50
1157         Y       Fell overboard     26
1116         Y               Diving     24
1402         Y            Swimming      21
1365         Y           Snorkeling     19
1383         Y             Standing     16
1068         Y        Body boarding     16
1520         Y               Wading     15
1321         Y         Scuba diving     15
1300         Y         Pearl diving     10
1507         Y       Treading water      7
1153         Y  Fell into the water      7
1070         Y         Body surfing      6
1380         Y        Sponge diving      6
1222         Y          Free diving      6
1330         Y         Sea Disaster      4


In [501]:
# Group by state and fatal status, Location?????
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Location']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)

  Fatal Y/N                                           Location  Count
1         Y  <bound method Series.unique of 0              ...   1480


In [400]:
df["Location"]

0       <bound method Series.unique of 0              ...
1       <bound method Series.unique of 0              ...
2       <bound method Series.unique of 0              ...
3       <bound method Series.unique of 0              ...
4       <bound method Series.unique of 0              ...
                              ...                        
7015    <bound method Series.unique of 0              ...
7016    <bound method Series.unique of 0              ...
7017    <bound method Series.unique of 0              ...
7018    <bound method Series.unique of 0              ...
7019    <bound method Series.unique of 0              ...
Name: Location, Length: 7020, dtype: string

In [143]:
# Group by state and fatal status, Country
grouped_state_fatal = df.groupby(['Fatal Y/N', 'Country']).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_only = grouped_state_fatal[grouped_state_fatal['Fatal Y/N'] == 'Y']

# Sort by count and get top 20
top_20_fatal_states = fatal_only.sort_values(by='Count', ascending=False).head(20)

# Show results
print(top_20_fatal_states)
# AUSTRALIA, USA, South Africa are the top three dangerouse destinations while coutries like: GABON, SOLOMON ISLANDS / VANUATU, ALGERIA restried less accedents 

    Fatal Y/N           Country  Count
139         Y         AUSTRALIA    307
261         Y               USA    198
241         Y      SOUTH AFRICA    109
223         Y  PAPUA NEW GUINEA     58
208         Y            MEXICO     50
148         Y            BRAZIL     39
227         Y       PHILIPPINES     37
221         Y             OTHER     35
229         Y           REUNION     29
217         Y       NEW ZEALAND     26
162         Y              CUBA     26
215         Y     NEW CALEDONIA     25
169         Y              FIJI     23
212         Y        MOZAMBIQUE     22
183         Y             INDIA     21
222         Y            PANAMA     18
191         Y           JAMAICA     17
166         Y             EGYPT     17
182         Y         HONG KONG     17
185         Y         INDONESIA     16


In [505]:
# Group by Fatal Y/N, Age_group, and Sex  
grouped_age_sex_fatal = df.groupby(['Fatal Y/N','Age_group', 'Sex', ]).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_age_sex = grouped_age_sex_fatal[grouped_age_sex_fatal['Fatal Y/N'] == 'Y']

# Sort by count
fatal_age_sex = fatal_age_sex.sort_values(by='Count', ascending=False)

# Show results
print(fatal_age_sex)
# Male adults are on top target of the sharks

   Fatal Y/N Age_group Sex  Count
19         Y   Unknown   M    652
11         Y     Adult   M    388
17         Y      Teen   M    165
10         Y     Adult   F     46
18         Y   Unknown   F     41
13         Y     Child   M     31
16         Y      Teen   F     22
15         Y    Senior   M     13
14         Y    Senior   F      8
12         Y     Child   F      5


In [135]:
# Group by Fatal Y/N Age_group, Sex, and Country
grouped_age_sex_loc_fatal = df.groupby(['Fatal Y/N','Age_group', 'Sex', 'Country' ]).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_age_sex_loc = grouped_age_sex_loc_fatal[grouped_age_sex_loc_fatal['Fatal Y/N'] == 'Y']

# Sort by count
fatal_age_sex_loc = fatal_age_sex_loc.sort_values(by='Count', ascending=False).head(10)

# Show results
print(fatal_age_sex_loc)
# France can be save for the females

    Fatal Y/N Age_group Sex           Country  Count
369         Y     Adult   M         AUSTRALIA    122
635         Y   Unknown   M               USA    103
537         Y   Unknown   M         AUSTRALIA     98
468         Y      Teen   M         AUSTRALIA     52
422         Y     Adult   M               USA     48
414         Y     Adult   M      SOUTH AFRICA     44
618         Y   Unknown   M      SOUTH AFRICA     32
604         Y   Unknown   M  PAPUA NEW GUINEA     31
602         Y   Unknown   M             OTHER     25
505         Y      Teen   M      SOUTH AFRICA     23


In [141]:
# Group by Fatal Y/N Age_group,Date_Range_Label, Sex, Activity , and Country
grouped_age_sex_loc_fatal = df.groupby(['Fatal Y/N','Date_Range_Label', 'Sex', 'Country', "Activity" ]).size().reset_index(name='Count')

# Filter only fatal incidents
fatal_age_sex_loc = grouped_age_sex_loc_fatal[grouped_age_sex_loc_fatal['Fatal Y/N'] == 'Y']

# Sort by count
fatal_age_sex_loc = fatal_age_sex_loc.sort_values(by='Count', ascending=False).head(5)

# Show results
print(fatal_age_sex_loc)
# Swimming in USA in summer is very dangerouse while it is very risky in Australia during summer and Spring

     Fatal Y/N Date_Range_Label Sex       Country  Activity  Count
2249         Y   21-06 to 21-09   M           USA  Swimming     25
2480         Y   21-12 to 21-03   M     AUSTRALIA  Swimming     22
2308         Y   21-09 to 21-12   M     AUSTRALIA  Swimming     21
2605         Y   21-12 to 21-03   M  SOUTH AFRICA  Swimming     21
2454         Y   21-12 to 21-03   M     AUSTRALIA   Bathing     14


In [157]:
# Total and fatal attacks
total_attacks = df['Country'].value_counts()
fatal_attacks = df[df['Fatal Y/N'] == 'Y']['Country'].value_counts()

# Summary table
summary = pd.DataFrame({
    'Total_Attacks': total_attacks,
    'Fatal_Attacks': fatal_attacks
})
summary['Fatal_Attacks'] = summary['Fatal_Attacks'].fillna(0)
summary['Fatality_Rate (%)'] = (summary['Fatal_Attacks'] / summary['Total_Attacks']) * 100

# Top countries with many attacks, but low fatality rate
top_attack_countries = summary.sort_values(by='Total_Attacks', ascending=False).head(10)
top_attack_countries = top_attack_countries.sort_values(by='Fatality_Rate (%)')

print(top_attack_countries)


                  Total_Attacks  Fatal_Attacks  Fatality_Rate (%)
Country                                                          
USA                        2561          198.0           7.731355
BAHAMAS                     139           15.0          10.791367
NEW ZEALAND                 146           26.0          17.808219
SOUTH AFRICA                598          109.0          18.227425
ITALY                        72           14.0          19.444444
AUSTRALIA                  1502          307.0          20.439414
BRAZIL                      122           39.0          31.967213
OTHER                        83           35.0          42.168675
PAPUA NEW GUINEA            136           58.0          42.647059
MEXICO                      107           50.0          46.728972
